In [3]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.multitest import multipletests
from sklearn.model_selection import train_test_split

In [2]:
pip install lifelines

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.4/409.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 82.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 95.3 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4032 sha256=3f27f6c5c450701111bf42dd014a9bf076d528918de27c34c6da4825f40c8c59
  Stored in directory: /tmp/pip-ephem-wheel-cache-n28ve38k/wheels/16/a2/b6/582cfdfbeeccd469504a01af3bb952fd9e7eccba40995eafea
Successfully built autograd-gamma
Note: you may need to restart the kernel to use updated packages.


In [5]:
full_data = pd.read_csv('Full Data.txt', index_col = [0])

In [6]:
full_data

,gender,ajcc_pathologic_tumor_stage,tumor_status,DSS,DSS.time,TP53,TTN,FAT1,MUC16,CSMD3,...,NIPBL,CEACAM5,CEACAM6,CLCA4,EGFR,Missense_Mutation,Nonsense_Mutation,Nonstop_Mutation,Splice_Site,age
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-4P-AA8J,1,0,0,0.0,0.279452,822.678,1157.3300,1620.71,1004.4700,1.2325,...,1595.440,56.0776,423.3550,502.2340,722.847,92,8,0,2,1
TCGA-BA-4074,1,0,1,1.0,1.265753,1805.510,103.6870,6415.45,17.3422,0.3854,...,1605.890,153.3820,275.9330,12.7176,1267.900,106,8,0,1,1
TCGA-BA-4075,1,1,1,1.0,0.775342,383.760,295.9180,4162.40,1.0720,36.9824,...,907.410,18.2232,45.5581,10.1836,4833.430,113,10,0,4,0
TCGA-BA-4077,0,0,1,1.0,3.106849,1929.260,207.3870,7246.87,432.6270,0.0000,...,1174.690,2392.6900,2437.9200,720.8850,2371.940,234,20,0,6,0
TCGA-BA-5149,1,0,1,1.0,2.208219,165.298,347.9540,8848.73,110.5410,0.3422,...,1798.770,37.3032,447.6390,5.4757,6934.280,96,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-UF-A7JO,0,0,1,1.0,1.728767,1604.880,176.1930,1881.36,347.7040,2.8008,...,854.256,1498.8500,2014.6000,82.8248,530.911,135,9,0,4,1
TCGA-UF-A7JS,1,0,1,1.0,1.863014,2457.400,1379.4300,1339.31,243.6470,19.4320,...,982.063,358.7440,294.4690,32.8849,246.637,50,1,0,1,1
TCGA-UF-A7JT,0,0,1,1.0,2.720548,1684.850,149.5190,3430.74,213.8530,14.2857,...,1009.520,49.7835,209.5240,200.4330,790.905,152,10,0,3,1


# Log-rank test

In [7]:
def quantile_gene(df, x):
    q1 = df[x].quantile(0.1)
    q3 = df[x].quantile(0.9)
    df[x + 'binary'] = df[x].apply(lambda x: 0 if x <= q1 else 1 if x >= q3 else np.nan)

In [8]:
def logrank(df, lis,time):
    dic ={}
    data = df[df['DSS.time'] <= time]
    for i in lis:
        quantile_gene(data, i)
        low_expression = data[data[i + 'binary'] == 0]
        high_expression =  data[data[i +'binary'] == 1]
        # logrank
        results = logrank_test(high_expression['DSS.time'], low_expression['DSS.time'],
                       high_expression['DSS'], low_expression['DSS'])
        z_value = results.test_statistic
        if z_value > 1.96 or z_value < -1.96:
            dic[i] = z_value
    # return a dictionary of name and z value
    return dic

In [9]:
gene_expression = list(full_data.drop(columns = ['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status', 'DSS' ,
                                      'DSS.time', 'Missense_Mutation', 'Nonsense_Mutation', 
                                      'Nonstop_Mutation', 'Splice_Site', 'age']).columns)

In [10]:
# for gene expression
logrank(full_data,gene_expression, 2) # example: use full data and 2 years as limit

{'TP53': 2.120617453561617,
 'TTN': 2.143271855534282,
 'CDKN2A': 3.1481755529943243,
 'PIK3CA': 4.680473121342188,
 'CASP8': 3.389085031356897,
 'XIRP2': 3.1703821360088784,
 'CUBN': 4.046813671678665,
 'MUC17': 3.646690183663279,
 'MUC5B': 2.0729664690136795,
 'PCDH15': 2.6959108383640076,
 'PCDH11X': 2.5834320206641146,
 'AHNAK2': 2.214970513627507,
 'FMN2': 2.615400302580672,
 'PKHD1': 2.104351556763302,
 'CDH10': 3.070610713909658,
 'EP300': 3.1299077752441216,
 'ANK2': 2.7759016074401215,
 'RYR1': 2.9064141544880564,
 'VCAN': 2.7374238783551403,
 'EGFR': 3.7419164805770855}

In [302]:
#select top n genes by sort z values as features from full_data

In [11]:
mutation = ['Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'Splice_Site']

In [12]:
# for mutation
logrank(full_data,mutation, 2) # example: use full data and 2 years as limit

{'Missense_Mutation': 2.2088500784074974,
 'Nonsense_Mutation': 1.9839793008189441,
 'Splice_Site': 5.71020812335661}

# Cox model

In [13]:
def coxfitter(data, duration, event):
    # Define the CoxPHFitter object
    cph = CoxPHFitter()
    # Fit the model to your data
    cph.fit(data, duration_col= duration, event_col=event)
    summary = cph.summary
    return summary

In [14]:
def cox_model(df, time, lis):
    data = df[df['DSS.time'] <= time]
    signi = []
    pvals = []
    pvals_correct = []
    for i in lis:
        summary = coxfitter(data[[i ,'gender','ajcc_pathologic_tumor_stage', 
                                   'tumor_status', 'age', 'DSS.time', 'DSS']], 'DSS.time', 'DSS')
        p_values = summary["p"]
        rejected, pvals_corrected = multipletests(p_values, method='fdr_bh')[:2]
        summary['p_corrected'] = pvals_corrected
        p = summary.loc[i, "p"]
        p_correct = summary.loc[i, "p_corrected"]
        pvals.append(p)
        pvals_correct.append(p_correct)
        signi.append(rejected[0])
    features = [col for col in lis]  # list of features
    # Create a DataFrame with the results
    results = pd.DataFrame({'Features': features,'p': pvals,'p_corrected': pvals_correct, 'significant': signi})
    return results[results['significant'] == True]


In [15]:
# gene expression level
cox_model(full_data, 2, gene_expression )# example: use full data and 2 years as limit

,Features,p,p_corrected,significant
19,CASP8,2.297870e-03,1.148935e-02,True
42,DNAH9,3.421607e-03,1.710804e-02,True
43,DNAH11,3.413931e-03,1.706965e-02,True
46,LRP2,1.491141e-38,7.455703e-38,True
54,RP1,2.284010e-03,1.142005e-02,True
59,DNAH7,7.134398e-03,3.567199e-02,True
83,MDN1,8.718074e-03,4.359037e-02,True


In [16]:
# mutation
cox_model(full_data, 2, mutation)# example: use full data and 2 years as limit

,Features,p,p_corrected,significant
0,Missense_Mutation,4.747304e-08,2.373652e-07,True
1,Nonsense_Mutation,3.141682e-04,1.570841e-03,True
2,Nonstop_Mutation,8.020092e-03,4.010046e-02,True
3,Splice_Site,3.520589e-05,1.760294e-04,True


# Classification

In [318]:
#select top n mutations by z values as features columns from full_data

In [ ]:
def classifcation_expression(df):
    # get features you select
    X = df[[...]]
    y = df['DSS']

    # Split the dataset into training and testing datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    # add classification
    # choose any classfication to do prediction
    ...
    return ...

In [18]:
pip install deepsurv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Created wheel for deepsurv: filename=deepsurv-0.1.0-py3-none-any.whl size=11146 sha256=1fd774899a01f1d4696b81ec9205f3656f113ee92085b7a4976416b0e23643d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-rpkifis1/wheels/8b/35/64/cb4fa77f7de25e6133c42cc1749ecd28b82131fcc937ab37ee
Successfully built deepsurv
Note: you may need to restart the kernel to use updated packages.


In [21]:
import numpy as np
from deepsurv import deep_surv
from deepsurv.datasets import load_rgbsg
from deepsurv.utils import train_test_split
X = full_data[['gender', 'ajcc_pathologic_tumor_stage', 'tumor_status','TP53',	'TTN',	'FAT1',	'MUC16','CLCA4','EGFR','Missense_Mutation',	'Nonsense_Mutation',	'Nonstop_Mutation',	'Splice_Site',	'age']]  # Replace with the actual feature columns
T = full_data['DSS.time']  # Replace with the column representing time-to-event
E = full_data['DSS']  # Replace with the column representing event indicator


# Split the dataset into training and testing sets
X_train, X_test, T_train, T_test, E_train, E_test = train_test_split(
    X, T, E, test_size=0.2, seed=0
)

# Define the DeepSurv model architecture
hyperparams = {'L2_reg': 1e-4, 'batch_norm': True, 'dropout': 0.2}
architecture = [
    {'activation': 'relu', 'num_units': 100},
    {'activation': 'relu', 'num_units': 100},
]

# Train the DeepSurv model
model = deep_surv.DeepSurv(architecture=architecture, hyperparams=hyperparams)
model.fit(X_train, T_train, E_train, num_epochs=100)

# Evaluate the model
c_index = model.concordance_index(X_test, T_test, E_test)
#print("Concordance Index:", c_index)


AttributeError: module 'pydot' has no attribute 'find_graphviz'